In [1]:
from nn import * 
from testing_models import test_model_nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
input_size = 28 * 28  # Flattened image size (28x28)
num_classes = 93  # Number of classes (modify if needed)
learning_rate = 0.001
num_epochs = 20
batch_size = 64
set_seed(42)

dataset = AlphaNumDataset(csv_dir="ascii_file_counts.csv", data_dir="train")
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataset_test = AlphaNumDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
val_dataset = AlphaNumDataset(csv_dir="ascii_file_counts.csv", data_dir="validation")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)




In [3]:
# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

Epoch [1/20], Train Loss: 2.7712, Validation Loss: 1.8609, Validation Accuracy: 49.62%
Validation loss improved! Saving model...
Epoch [2/20], Train Loss: 1.6891, Validation Loss: 1.7061, Validation Accuracy: 52.11%
Validation loss improved! Saving model...
Epoch [3/20], Train Loss: 1.4046, Validation Loss: 1.3880, Validation Accuracy: 59.94%
Validation loss improved! Saving model...
Epoch [4/20], Train Loss: 1.2490, Validation Loss: 1.3412, Validation Accuracy: 61.04%
Validation loss improved! Saving model...
Epoch [5/20], Train Loss: 1.1413, Validation Loss: 1.1856, Validation Accuracy: 65.63%
Validation loss improved! Saving model...
Epoch [6/20], Train Loss: 1.0552, Validation Loss: 1.2853, Validation Accuracy: 62.91%
Validation loss did not improve. Patience: 1/3
Epoch [7/20], Train Loss: 0.9994, Validation Loss: 1.0700, Validation Accuracy: 68.27%
Validation loss improved! Saving model...
Epoch [8/20], Train Loss: 0.9415, Validation Loss: 0.9721, Validation Accuracy: 71.38%
Valid

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

Epoch [1/20], Train Loss: 2.7712, Validation Loss: 1.8609, Validation Accuracy: 49.62%
Validation loss improved! Saving model...
Epoch [2/20], Train Loss: 1.6891, Validation Loss: 1.7061, Validation Accuracy: 52.11%
Validation loss improved! Saving model...
Epoch [3/20], Train Loss: 1.4046, Validation Loss: 1.3880, Validation Accuracy: 59.94%
Validation loss improved! Saving model...
Epoch [4/20], Train Loss: 1.2490, Validation Loss: 1.3412, Validation Accuracy: 61.04%
Validation loss improved! Saving model...
Epoch [5/20], Train Loss: 1.1413, Validation Loss: 1.1856, Validation Accuracy: 65.63%
Validation loss improved! Saving model...
Epoch [6/20], Train Loss: 1.0552, Validation Loss: 1.2853, Validation Accuracy: 62.91%
Validation loss did not improve. Patience: 1/3


KeyboardInterrupt: 

In [ ]:
torch.save(model, "FeedForward.pth")

In [4]:
dataset_test = AlphaNumDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
test_model_nn(model, data_loader_test, device, criterion)

(0.7323994581800591, 0.7393526405451448, array([[ 21,   0,   0, ...,   0,   5,   0],
       [  0,  53,   0, ...,   0,   0,   0],
       [  0,   0,  13, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   5,   0],
       [  0,   0,   0, ...,   0,   0, 994]]), 0.7468134050488362, 0.7393526405451448)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pac

(0.7323994581800591,
 0.7393526405451448,
 0.7468134050488362,
 0.7393526405451448)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

Epoch [1/20], Train Loss: 2.3924, Validation Loss: 1.3834, Validation Accuracy: 61.52%
Validation loss improved! Saving model...
Epoch [2/20], Train Loss: 1.1677, Validation Loss: 1.0250, Validation Accuracy: 70.37%
Validation loss improved! Saving model...
Epoch [3/20], Train Loss: 0.9171, Validation Loss: 0.8584, Validation Accuracy: 74.36%
Validation loss improved! Saving model...
Epoch [4/20], Train Loss: 0.7806, Validation Loss: 0.7628, Validation Accuracy: 76.05%
Validation loss improved! Saving model...
Epoch [5/20], Train Loss: 0.6966, Validation Loss: 0.7090, Validation Accuracy: 78.14%
Validation loss improved! Saving model...
Epoch [6/20], Train Loss: 0.6422, Validation Loss: 0.6771, Validation Accuracy: 78.42%
Validation loss improved! Saving model...
Epoch [7/20], Train Loss: 0.5973, Validation Loss: 0.6303, Validation Accuracy: 79.81%
Validation loss improved! Saving model...
Epoch [8/20], Train Loss: 0.5647, Validation Loss: 0.5993, Validation Accuracy: 80.68%
Validation

In [5]:
test_model(model, data_loader_test, device, criterion)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: too many values to unpack (expected 3)

In [7]:
torch.save(model,"cnn.pth")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

Epoch [1/20], Train Loss: 2.5150, Validation Loss: 1.3859, Validation Accuracy: 58.39%
Validation loss improved! Saving model...
Epoch [2/20], Train Loss: 1.0294, Validation Loss: 0.8222, Validation Accuracy: 74.00%
Validation loss improved! Saving model...
Epoch [3/20], Train Loss: 0.7244, Validation Loss: 0.6830, Validation Accuracy: 77.65%
Validation loss improved! Saving model...
Epoch [4/20], Train Loss: 0.6022, Validation Loss: 0.6135, Validation Accuracy: 79.57%
Validation loss improved! Saving model...
Epoch [5/20], Train Loss: 0.5277, Validation Loss: 0.5538, Validation Accuracy: 81.26%
Validation loss improved! Saving model...
Epoch [6/20], Train Loss: 0.4738, Validation Loss: 0.5067, Validation Accuracy: 82.84%
Validation loss improved! Saving model...
Epoch [7/20], Train Loss: 0.4351, Validation Loss: 0.4740, Validation Accuracy: 83.70%
Validation loss improved! Saving model...
Epoch [8/20], Train Loss: 0.3993, Validation Loss: 0.4708, Validation Accuracy: 84.12%
Validation

In [ ]:
test_model(model, data_loader_test, device, criterion)

In [ ]:
torch.save(model,"cnn_lstm.pth")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNTransformer(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
#train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
test_model(model, data_loader_test, device, criterion)

In [ ]:
torch.save(model,"transformer.pth")

In [4]:
test = torch.load("transformer.pth")
test_model(test,data_loader_test, device, criterion)

/var/folders/92/3_42v5w111v7m641kr368p5c0000gn/T/ipykernel_36113/2581909732.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test = torch.load("transformer.pth")


Model: <class 'nn.CNNTransformer'>
Accuracy: 0.8664
Precision: 0.8689
Recall: 0.8664
F1 Score: 0.8640
Confusion Matrix:
[[  40    0    0 ...    0    0    0]
 [   0   45    0 ...    0    0    0]
 [   0    0   19 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   1    0    0 ...    0    9    0]
 [   0    0    0 ...    0    0 1010]]




/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.391109123322017,
 86.64303144712004,
 0.8639561695556224,
 array([[  40,    0,    0, ...,    0,    0,    0],
        [   0,   45,    0, ...,    0,    0,    0],
        [   0,    0,   19, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    0,    0, ...,    0,    9,    0],
        [   0,    0,    0, ...,    0,    0, 1010]]))